In [1]:
import pandas as pd
import numpy as np

import string
from re import sub

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.preprocessing import LabelEncoder
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix


In [3]:
y_train, x_b, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, x_tb, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))

In [4]:
import tensorflow as tf

from time import time

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



In [15]:
y_train_1000 = y_train[slice(0, len(y_train), 10)]
x_train_1000 = x_train[slice(0, len(x_train), 10)]
# y_test_1000 = y_test[slice(0, len(y_test), 40)]
# x_test_1000 = x_test[slice(0, len(x_test), 40)]

y_valid = y_train[slice(1, len(y_train), 40)]
x_valid = x_train[slice(1, len(x_train), 40)]

In [6]:
classes = [x for x in range(1, 15)]
classes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [17]:
x_train_1000.shape

(56000, 1000)

In [16]:
y_train_1000_np = []

for y in y_train_1000:
    l = [0 for x in range(14)]
    l[y-1] = 1
    y_train_1000_np.append(l)
    
y_train_1000_np = np.asarray(y_train_1000_np)
y_train_1000_np.shape

(56000, 14)

In [18]:
fname = "dbpedia_csv/" + "_train_10_percent.pickle"
pickle.dump((y_train_1000_np, x_train_1000), open(fname, "wb"), protocol=pickle.HIGHEST_PROTOCOL)




In [8]:
callback = EarlyStopping(monitor='val_loss',patience=2, min_delta=0.0001)

checkpoint = ModelCheckpoint("best_model", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [14]:
model_l1 = Sequential()
model_l1.add(Embedding(14000, 1000, input_length=x_train_1000.shape[1]))
model_l1.add(Conv1D(128, 5, activation='relu'))
model_l1.add(GlobalMaxPooling1D()) # added
model_l1.add(Dense(128, activation='relu'))
model_l1.add(Dropout(0.1))
model_l1.add(Dense(32, activation='relu'))
model_l1.add(Dense(14, activation='softmax')) 
model_l1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_l1.summary()

history_l1 = model_l1.fit(x_train_1000, y_train_1000_np, epochs=20, verbose=1, validation_split=0.2, batch_size=128
                   ,callbacks=[reduce,callback,checkpoint])

score_l1 = model_l1.evaluate(x_test, y_test, verbose=1)
print('Test score:', score_l1[0])
print('Test accuracy:', score_l1[1])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1000, 1000)        14000000  
                                                                 
 conv1d_3 (Conv1D)           (None, 996, 128)          640128    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                      

INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets


88/88 [==============================] - 82s 758ms/step - loss: 2.5049 - accuracy: 0.0969 - val_loss: 5.3143 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/20
88/88 [==============================] - ETA: 0s - loss: 2.4604 - accuracy: 0.0904
Epoch 2: val_loss did not improve from 5.31428
88/88 [==============================] - 60s 677ms/step - loss: 2.4604 - accuracy: 0.0904 - val_loss: 5.9506 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/20
31/88 [=========>....................] - ETA: 36s - loss: 2.4574 - accuracy: 0.0799

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_3/conv1d_3/Conv1D/Conv2DBackpropInput' defined at (most recent call last):
    File "C:\Users\Ana\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Ana\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Ana\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Ana\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Ana\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Ana\anaconda3\lib\asyncio\base_events.py", line 601, in run_forev